## Dependecies

In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine

 ## Store CSV into DataFrame

In [2]:
# Read the SNIIV CSV

sniiv_file = "Resources/sniiv.csv"
sniiv_df = pd.read_csv(sniiv_file, encoding='latin-1')
sniiv_df.head()

,Año,Mes acumulado,Clave Entidad INEGI,Entidad Federativa,Clave Municipio INEGI,Municipio,Organismo,Modalidad,Destino del Credito,Tipo de Credito,Genero,Rango de Edad,Rango de Ingresos UMA,Valor de la Vivienda,Acciones,Monto
0,2020,1,17,Morelos,18,Temixco,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,29 o menos,2.6 o menos,Económica,1,"17,209.00"
1,2020,1,4,Campeche,3,Carmen,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,30 a 59,4.01 a 6.00,Económica,3,"158,963.00"
2,2020,1,9,Ciudad de México,2,Azcapotzalco,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,29 o menos,2.6 o menos,Económica,1,"19,516.00"
3,2020,1,13,Hidalgo,51,Mineral de la Reforma,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,30 a 59,4.01 a 6.00,Económica,1,"57,534.00"
4,2020,1,2,Baja California,4,Tijuana,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Mujer,30 a 59,2.61 a 4.00,Económica,7,"340,331.00"


## SNIIV - Data Transforming

In [3]:
# Get a list of all of our columns for easy reference.

sniiv_df.columns

Index(['Año', 'Mes acumulado', 'Clave Entidad INEGI', 'Entidad Federativa',
       'Clave Municipio INEGI', 'Municipio', 'Organismo', 'Modalidad',
       'Destino del Credito', 'Tipo de Credito', 'Genero', 'Rango de Edad',
       'Rango de Ingresos UMA', 'Valor de la Vivienda', 'Acciones', 'Monto'],
      dtype='object')

In [4]:
# Reduce DF to allocate just the needed columns.

sniiv_analysis_df = sniiv_df.loc[:, ["Clave Entidad INEGI","Clave Municipio INEGI", "Genero", "Rango de Edad","Monto"]]
sniiv_analysis_df

,Clave Entidad INEGI,Clave Municipio INEGI,Genero,Rango de Edad,Monto
0,17,18,Hombre,29 o menos,"17,209.00"
1,4,3,Hombre,30 a 59,"158,963.00"
2,9,2,Hombre,29 o menos,"19,516.00"
3,13,51,Hombre,30 a 59,"57,534.00"
4,2,4,Mujer,30 a 59,"340,331.00"
...,...,...,...,...,...
355369,19,31,Hombre,30 a 59,"88,527.00"
355370,21,142,Mujer,30 a 59,"105,777.00"
355371,16,68,Hombre,29 o menos,"66,281.00"
355372,8,21,Hombre,30 a 59,"180,938.00"


In [5]:
# Replace the values of columns "Genero" & "Rango de Edad" with keys (https://datatofish.com/replace-values-pandas-dataframe/)

sniiv_analysis_df["Genero"] = sniiv_analysis_df["Genero"].replace(["Hombre","Mujer", "No disponible"],[0,1,2])

sniiv_analysis_df["Rango de Edad"] = sniiv_analysis_df["Rango de Edad"].replace(["29 o menos","30 a 59","60 o más", "No disponible"],[0,1,2,3])

sniiv_analysis_df

,Clave Entidad INEGI,Clave Municipio INEGI,Genero,Rango de Edad,Monto
0,17,18,0,0,"17,209.00"
1,4,3,0,1,"158,963.00"
2,9,2,0,0,"19,516.00"
3,13,51,0,1,"57,534.00"
4,2,4,1,1,"340,331.00"
...,...,...,...,...,...
355369,19,31,0,1,"88,527.00"
355370,21,142,1,1,"105,777.00"
355371,16,68,0,0,"66,281.00"
355372,8,21,0,1,"180,938.00"


In [6]:
# Convert Claves to string to be able to concatenate
sniiv_analysis_df["Clave Entidad INEGI"] = sniiv_analysis_df["Clave Entidad INEGI"].astype(str)
sniiv_analysis_df["Clave Municipio INEGI"] = sniiv_analysis_df["Clave Municipio INEGI"].astype(str)
sniiv_analysis_df

,Clave Entidad INEGI,Clave Municipio INEGI,Genero,Rango de Edad,Monto
0,17,18,0,0,"17,209.00"
1,4,3,0,1,"158,963.00"
2,9,2,0,0,"19,516.00"
3,13,51,0,1,"57,534.00"
4,2,4,1,1,"340,331.00"
...,...,...,...,...,...
355369,19,31,0,1,"88,527.00"
355370,21,142,1,1,"105,777.00"
355371,16,68,0,0,"66,281.00"
355372,8,21,0,1,"180,938.00"


In [7]:
# Check what type are each variable.
sniiv_analysis_df.dtypes

Clave Entidad INEGI      object
Clave Municipio INEGI    object
Genero                    int64
Rango de Edad             int64
Monto                    object
dtype: object

In [8]:
# Making a coy of Clave Municipio INEGI to be able to concatenate.
new = sniiv_analysis_df["Clave Municipio INEGI"].copy()

# Concatenating Clave Entidad INEGI  & Clave Municipio INEGI with name column ID
sniiv_analysis_df["Clave Entidad INEGI"] = sniiv_analysis_df["Clave Entidad INEGI"].str.cat(new,sep="-")
sniiv_analysis_df

,Clave Entidad INEGI,Clave Municipio INEGI,Genero,Rango de Edad,Monto
0,17-18,18,0,0,"17,209.00"
1,4-3,3,0,1,"158,963.00"
2,9-2,2,0,0,"19,516.00"
3,13-51,51,0,1,"57,534.00"
4,2-4,4,1,1,"340,331.00"
...,...,...,...,...,...
355369,19-31,31,0,1,"88,527.00"
355370,21-142,142,1,1,"105,777.00"
355371,16-68,68,0,0,"66,281.00"
355372,8-21,21,0,1,"180,938.00"


In [9]:
# Rename the columns a create new data frame.

sniiv_adjusted_df = sniiv_analysis_df.rename(columns = {"Clave Entidad INEGI":"Clave_Municipio", "Genero":"ID_Genero", 
                                                                 "Rango de Edad":"ID_Rango_Edad","Monto":"Prestamo"}, inplace=False).copy()

sniiv_adjusted_df

,Clave_Municipio,Clave Municipio INEGI,ID_Genero,ID_Rango_Edad,Prestamo
0,17-18,18,0,0,"17,209.00"
1,4-3,3,0,1,"158,963.00"
2,9-2,2,0,0,"19,516.00"
3,13-51,51,0,1,"57,534.00"
4,2-4,4,1,1,"340,331.00"
...,...,...,...,...,...
355369,19-31,31,0,1,"88,527.00"
355370,21-142,142,1,1,"105,777.00"
355371,16-68,68,0,0,"66,281.00"
355372,8-21,21,0,1,"180,938.00"


## SNIIV - Cleaned Table

In [10]:
# Select the final columns & Reset the index

sniiv_final_df = sniiv_adjusted_df[['Clave_Municipio', 'ID_Genero', 'ID_Rango_Edad', "Prestamo"]].copy()

sniiv_final_df = sniiv_final_df.set_index('Clave_Municipio')

sniiv_final_df

,ID_Genero,ID_Rango_Edad,Prestamo
Clave_Municipio,,,
17-18,0,0,"17,209.00"
4-3,0,1,"158,963.00"
9-2,0,0,"19,516.00"
13-51,0,1,"57,534.00"
2-4,1,1,"340,331.00"
...,...,...,...
19-31,0,1,"88,527.00"
21-142,1,1,"105,777.00"
16-68,0,0,"66,281.00"


## Genero - Table

In [11]:
# Create a Table of "Genero" according to the parameters of SNIIV data set.

data = {'ID': [0, 1, 2], 'Genero': ["Hombre", "Mujer", "No disponible"]}
gender_df = pd.DataFrame(data=data)

gender_df = gender_df.set_index('ID')

gender_df

,Genero
ID,
0,Hombre
1,Mujer
2,No disponible


## Rango Edad - Table

In [12]:
# Create a Table of "Rango Edad" according to the parameters of SNIIV data set.

data = {'ID': [0,1,2,3], 'Rango': ["29 o menos","30 a 59","60 o más", "No disponible"]}
rango_edad_df = pd.DataFrame(data=data)

rango_edad_df = rango_edad_df.set_index('ID')

rango_edad_df

,Rango
ID,
0,29 o menos
1,30 a 59
2,60 o más
3,No disponible


## Entidad - Table

In [13]:
# Read the INEGI CSV

inegi_file = "Resources/inegi.csv"
inegi_df = pd.read_csv(inegi_file)
inegi_df.head()

/Users/adrianmarceloflorespedraza/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


In [14]:
# Create a new DF which contains the Entidad Table.

entidad = inegi_df[(inegi_df['LOC'] == 0) & (inegi_df['MUN'] == 0) & (inegi_df['ENTIDAD'] != 0)]
entidad_table = entidad[['ENTIDAD', 'NOM_ENT']]
entidad_table = entidad_table.rename(columns={'ENTIDAD':'Clave_Entidad', 'NOM_ENT':'Entidad_Fed'})
entidad_table = entidad_table.set_index('Clave_Entidad')
entidad_table

,Entidad_Fed
Clave_Entidad,
1,Aguascalientes
2,Baja California
3,Baja California Sur
4,Campeche
5,Coahuila de Zaragoza
6,Colima
7,Chiapas
8,Chihuahua
9,Ciudad de México


## Municipio - Table

In [15]:
# Select certain columns

new_inegi_df = inegi_df[['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'POBTOT', 'POBFEM', 'POBMAS', 'P_60YMAS', 'PEA', 'PEA_F', 'PEA_M',\
                        'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TOTHOG', 'HOGJEF_F', 'HOGJEF_M', 'VIVTOT', 'TVIVHAB',\
                       'PROM_OCUP', 'PRO_OCUP_C']].copy()
new_inegi_df.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,POBTOT,POBFEM,POBMAS,P_60YMAS,...,POCUPADA,POCUPADA_F,POCUPADA_M,TOTHOG,HOGJEF_F,HOGJEF_M,VIVTOT,TVIVHAB,PROM_OCUP,PRO_OCUP_C
0,0,Total nacional,0,Total nacional,0,Total nacional,126014024,64540634,61473390,15142976,...,61121324,25137019,35984305,35219141,11474983,23744158,43903443,35233462,3.56,0.96
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,250354,96869,153485,37383,...,127469,32906,94563,62516,8292,54224,101390,62767,3.18,1.03
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,147125,61324,85801,21277,...,68215,18365,49850,36568,6620,29948,61435,36676,3.42,1.03
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,1425607,728924,696683,145376,...,692983,289268,403715,386445,119453,266992,463972,386671,3.68,0.89
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,3697,1510,2187,448,...,2060,497,1563,807,123,684,1368,813,3.89,1.02


In [16]:
# Securing Clave_Entidad
new_inegi_df['Clave_Entidad'] = new_inegi_df['ENTIDAD']

# Adding new ID

# Convert claves to string to be able to concatenate
new_inegi_df['ENTIDAD'] = new_inegi_df['ENTIDAD'].astype(str)
new_inegi_df['MUN'] = new_inegi_df['MUN'].astype(str)

# Making a copy of clave entidad municipio to be able to concatenate
new = new_inegi_df['MUN'].copy()

# Concatenating clave Entidad & MUN
new_inegi_df['ENTIDAD'] = new_inegi_df['ENTIDAD'].str.cat(new, sep="-")

new_inegi_df.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,POBTOT,POBFEM,POBMAS,P_60YMAS,...,POCUPADA_F,POCUPADA_M,TOTHOG,HOGJEF_F,HOGJEF_M,VIVTOT,TVIVHAB,PROM_OCUP,PRO_OCUP_C,Clave_Entidad
0,0-0,Total nacional,0,Total nacional,0,Total nacional,126014024,64540634,61473390,15142976,...,25137019,35984305,35219141,11474983,23744158,43903443,35233462,3.56,0.96,0
1,0-0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,250354,96869,153485,37383,...,32906,94563,62516,8292,54224,101390,62767,3.18,1.03,0
2,0-0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,147125,61324,85801,21277,...,18365,49850,36568,6620,29948,61435,36676,3.42,1.03,0
3,1-0,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,1425607,728924,696683,145376,...,289268,403715,386445,119453,266992,463972,386671,3.68,0.89,1
4,1-0,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,3697,1510,2187,448,...,497,1563,807,123,684,1368,813,3.89,1.02,1


In [35]:
# Create a new DF which contains the Municipio Table.

new_inegi_df['MUN'] = new_inegi_df['MUN'].astype(int)
municipality = new_inegi_df[(new_inegi_df['LOC'] == 0) & (new_inegi_df['MUN'] != 0)]
municipality_table = municipality[['ENTIDAD', 'NOM_MUN', 'Clave_Entidad']]
municipality_table = municipality_table.rename(columns={'ENTIDAD':'Clave_Municipio', 'NOM_MUN':'Municipio'})
municipality_table = municipality_table.set_index('Clave_Municipio')
municipality_table.head()

,Municipio,Clave_Entidad
Clave_Municipio,,
1-1,Aguascalientes,1
1-2,Asientos,1
1-3,Calvillo,1
1-4,Cosío,1
1-5,Jesús María,1


## INEGI - Data Transforming

In [18]:
# Drop national totals and state totals rows.

new_inegi_df = new_inegi_df[new_inegi_df['LOC'] == 0]
new_inegi_df = new_inegi_df[new_inegi_df['MUN'] != 0]
new_inegi_df.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,POBTOT,POBFEM,POBMAS,P_60YMAS,...,POCUPADA_F,POCUPADA_M,TOTHOG,HOGJEF_F,HOGJEF_M,VIVTOT,TVIVHAB,PROM_OCUP,PRO_OCUP_C,Clave_Entidad
6,1-1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,948990,486917,462073,102987,...,205402,271100,266778,85732,181046,313256,266942,3.54,0.84,1
574,1-2,Aguascalientes,2,Asientos,0,Total del Municipio,51536,26275,25261,4697,...,7069,13945,12542,3191,9351,15353,12544,4.1,1.07,1
749,1-3,Aguascalientes,3,Calvillo,0,Total del Municipio,58250,29687,28563,7829,...,9342,16255,15546,5008,10538,20842,15556,3.74,0.97,1
918,1-4,Aguascalientes,4,Cosío,0,Total del Municipio,17000,8708,8292,1613,...,2401,4482,3937,889,3048,4845,3938,4.32,1.14,1
985,1-5,Aguascalientes,5,Jesús María,0,Total del Municipio,129929,65710,64219,9815,...,27147,37798,33214,9029,24185,41550,33229,3.91,0.95,1


In [19]:
# Converting columns to integer to be able to run calculations

new_inegi_df['POBTOT'] = new_inegi_df['POBTOT'].astype(int)
new_inegi_df['POBFEM'] = new_inegi_df['POBFEM'].astype(int)
new_inegi_df['POBMAS'] = new_inegi_df['POBMAS'].astype(int)
new_inegi_df['PEA'] = new_inegi_df['PEA'].astype(int)
new_inegi_df['PEA_F'] = new_inegi_df['PEA'].astype(int)
new_inegi_df['PEA_M'] = new_inegi_df['POCUPADA_F'].astype(int)
new_inegi_df['POCUPADA'] = new_inegi_df['POCUPADA_M'].astype(int)
new_inegi_df['POCUPADA_F'] = new_inegi_df['POCUPADA_M'].astype(int)
new_inegi_df['POCUPADA_M'] = new_inegi_df['POCUPADA_M'].astype(int)
new_inegi_df['TOTHOG'] = new_inegi_df['TOTHOG'].astype(int)
new_inegi_df['HOGJEF_F'] = new_inegi_df['HOGJEF_F'].astype(int)
new_inegi_df['HOGJEF_M'] = new_inegi_df['HOGJEF_M'].astype(int)
new_inegi_df['VIVTOT'] = new_inegi_df['VIVTOT'].astype(int)
new_inegi_df['TVIVHAB'] = new_inegi_df['TVIVHAB'].astype(int)
new_inegi_df['PROM_OCUP'] = new_inegi_df['PROM_OCUP'].astype(float)
new_inegi_df['PRO_OCUP_C'] = new_inegi_df['PRO_OCUP_C'].astype(float)

new_inegi_df.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,POBTOT,POBFEM,POBMAS,P_60YMAS,...,POCUPADA_F,POCUPADA_M,TOTHOG,HOGJEF_F,HOGJEF_M,VIVTOT,TVIVHAB,PROM_OCUP,PRO_OCUP_C,Clave_Entidad
6,1-1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,948990,486917,462073,102987,...,271100,271100,266778,85732,181046,313256,266942,3.54,0.84,1
574,1-2,Aguascalientes,2,Asientos,0,Total del Municipio,51536,26275,25261,4697,...,13945,13945,12542,3191,9351,15353,12544,4.10,1.07,1
749,1-3,Aguascalientes,3,Calvillo,0,Total del Municipio,58250,29687,28563,7829,...,16255,16255,15546,5008,10538,20842,15556,3.74,0.97,1
918,1-4,Aguascalientes,4,Cosío,0,Total del Municipio,17000,8708,8292,1613,...,4482,4482,3937,889,3048,4845,3938,4.32,1.14,1
985,1-5,Aguascalientes,5,Jesús María,0,Total del Municipio,129929,65710,64219,9815,...,37798,37798,33214,9029,24185,41550,33229,3.91,0.95,1


In [20]:
# Check what type are each variable

new_inegi_df.dtypes

ENTIDAD           object
NOM_ENT           object
MUN                int64
NOM_MUN           object
LOC                int64
NOM_LOC           object
POBTOT             int64
POBFEM             int64
POBMAS             int64
P_60YMAS          object
PEA                int64
PEA_F              int64
PEA_M              int64
POCUPADA           int64
POCUPADA_F         int64
POCUPADA_M         int64
TOTHOG             int64
HOGJEF_F           int64
HOGJEF_M           int64
VIVTOT             int64
TVIVHAB            int64
PROM_OCUP        float64
PRO_OCUP_C       float64
Clave_Entidad      int64
dtype: object

## INEGI - Cleaned Table

In [21]:
end_inegi_df = new_inegi_df[['ENTIDAD', 'POBTOT', 'POBFEM', 'POBMAS']].copy()
end_inegi_df = end_inegi_df.rename(columns = {'ENTIDAD':'Clave_Municipio', 'POBTOT': 'Pob_Total',\
                                             'POBFEM':'Pob_Fem', 'POBMAS':'Pob_Mas'}, inplace= False)

end_inegi_df = end_inegi_df.set_index('Clave_Municipio')

end_inegi_df.head( )

,Pob_Total,Pob_Fem,Pob_Mas
Clave_Municipio,,,
1-1,948990,486917,462073
1-2,51536,26275,25261
1-3,58250,29687,28563
1-4,17000,8708,8292
1-5,129929,65710,64219


##  Connect to local database

In [22]:
rds_connection_string = "postgres:amfp@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

##  Check for tables

In [23]:
engine.table_names()

['ENTIDAD', 'MUNICIPIO', 'INEGI', 'SNIIV', 'Genero', 'Rango_Edad']

In [24]:
# ENTIDAD
entidad_table.to_sql(name='ENTIDAD', con=engine, if_exists='append', index=True)

In [25]:
# MUNICIPALITY
municipality_table.to_sql(name='MUNICIPIO', con=engine, if_exists='append', index=True)

In [26]:
# INEGI
end_inegi_df.to_sql(name='INEGI', con=engine, if_exists='append', index=True)

In [27]:
# GENERO
gender_df.to_sql(name='Genero', con=engine, if_exists='append', index=True)

In [28]:
# RANGO EDAD
rango_edad_df.to_sql(name='Rango_Edad', con=engine, if_exists='append', index=True)

In [29]:
# SNIIV
sniiv_final_df.to_sql(name='SNIIV', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "SNIIV" violates foreign key constraint "SNIIV_Clave_Municipio_fkey"
DETAIL:  Key (Clave_Municipio)=(19-0) is not present in table "MUNICIPIO".

[SQL: INSERT INTO "SNIIV" ("Clave_Municipio", "ID_Genero", "ID_Rango_Edad", "Prestamo") VALUES (%(Clave_Municipio)s, %(ID_Genero)s, %(ID_Rango_Edad)s, %(Prestamo)s)]
[parameters: ({'Clave_Municipio': '17-18', 'ID_Genero': 0, 'ID_Rango_Edad': 0, 'Prestamo': '17,209.00'}, {'Clave_Municipio': '4-3', 'ID_Genero': 0, 'ID_Rango_Edad': 1, 'Prestamo': '158,963.00'}, {'Clave_Municipio': '9-2', 'ID_Genero': 0, 'ID_Rango_Edad': 0, 'Prestamo': '19,516.00'}, {'Clave_Municipio': '13-51', 'ID_Genero': 0, 'ID_Rango_Edad': 1, 'Prestamo': '57,534.00'}, {'Clave_Municipio': '2-4', 'ID_Genero': 1, 'ID_Rango_Edad': 1, 'Prestamo': '340,331.00'}, {'Clave_Municipio': '8-37', 'ID_Genero': 0, 'ID_Rango_Edad': 1, 'Prestamo': '905,599.00'}, {'Clave_Municipio': '19-6', 'ID_Genero': 1, 'ID_Rango

In [32]:
pd.read_sql_query('select * from "MUNICIPIO"', con=engine)

     Clave_Municipio              Municipio  Clave_Entidad
0                1-1         Aguascalientes              1
1                1-2               Asientos              1
2                1-3               Calvillo              1
3                1-4                  Cosío              1
4                1-5            Jesús María              1
...              ...                    ...            ...
2464           32-54          Villa Hidalgo             32
2465           32-55             Villanueva             32
2466           32-56              Zacatecas             32
2467           32-57               Trancoso             32
2468           32-58  Santa María de la Paz             32

[2469 rows x 3 columns]
